In [79]:
import boto3
import pandas as pd
from io import StringIO

In [80]:
s3 = boto3.resource('s3')

# Connecting to s3 bucket
bucket = s3.Bucket('xetra-1234')

In [81]:
# Inside the bucket there are a lot of folders, so we will connect to one of the with the name of foler as "2022-01-03"
bucket_obj1 = bucket.objects.filter(Prefix= '2022-01-03')

# We will create a 2nd bucket object as well so as to combine data available inside two buckets objects
    
    # Lets take the bucket object named "2022-01-04" so that when we combine data, we have 2 days worth of data combined in a single file.
bucket_obj2 = bucket.objects.filter(Prefix = "2022-01-04")

# Inside the bucket object again there are a lot of csv files, so we will create a variable with a list of all the files inside "2022-01-03"
    # objects1 is for all the files inside bucket_obj1
'''objects1 = [obj for obj in bucket_obj1]'''

    # objects2 is for all the files inside bucket_obj2
'''objects2 = [obj for obj in bucket_obj2]'''

# Or we can directly concatanate the list of two objects together

objects = [obj for obj in bucket_obj1] + [obj for obj in bucket_obj2]

In [82]:
# Checking whether joining the two lists is successful or not
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR10.csv'),
 s3.Object

* we can see that "objects" now contains the list of all the files from date "2022-01-03" and "2022-01-04"

In [83]:
# Lets get a list of column names of the data
csv_obj_init = bucket.Object(key = objects[0].key).get().get('Body').read().decode('utf-8')
data_init = StringIO(csv_obj_init)
df_init = pd.read_csv(data_init, delimiter = ',')

In [84]:
# chechking the culumn names in dataframe
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [85]:
# Creating an empty pandas dataframe and fixing the column names as per our dataset
df_merged = pd.DataFrame(columns=df_init.columns)

# Merging the data present in all the csv files into a single merged Dataset called df_merged using a for loop
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_merged = pd.concat([df_merged, df], ignore_index=True)

C:\Users\ankur\AppData\Local\Temp\ipykernel_24108\2990473283.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merged = pd.concat([df_merged, df], ignore_index=True)
C:\Users\ankur\AppData\Local\Temp\ipykernel_24108\2990473283.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merged = pd.concat([df_merged, df], ignore_index=True)
C:\Users\ankur\AppData\Local\Temp\ipykernel_24108\2990473283.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is depr

In [86]:
#lets check our merged dataset
df_merged

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414,2
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649,3
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011,36
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308,22
4,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239075,DE0007164600,SAP,SAP SE O.N.,Common stock,EUR,2505077,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000,3
239076,MT0000580101,M8G,MEDIA AND GAMES INV. EO 1,Common stock,EUR,2509636,2022-01-04,16:44,4.118,4.118,4.118,4.118,24,2
239077,DE000A0Z23Q5,ADN1,ADESSO SE INH O.N.,Common stock,EUR,2504440,2022-01-04,16:45,196.800,196.800,196.800,196.800,7,2
239078,DE000A3E5ES0,BIO0,BIOTEST AG ST O.N.Z.VERK.,Common stock,EUR,6973997,2022-01-04,20:30,43.000,43.000,43.000,43.000,0,1


In [87]:
# There are many columns which we do not need for further processing lets get rid of unwanted data
columns_to_included = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume']

In [88]:
# Filtering our merged dataframe to only those columns which we need for further process
df_merged = df_merged.loc[:,columns_to_included]
df_merged

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414
1,DE000A0DJ6J9,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649
2,DE000A0D6554,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011
3,DE000A0D9PT0,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308
4,DE000A0HN5C6,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897
...,...,...,...,...,...,...,...,...
239075,DE0007164600,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000
239076,MT0000580101,2022-01-04,16:44,4.118,4.118,4.118,4.118,24
239077,DE000A0Z23Q5,2022-01-04,16:45,196.800,196.800,196.800,196.800,7
239078,DE000A3E5ES0,2022-01-04,20:30,43.000,43.000,43.000,43.000,0


* df_merged now containes only those columns which we need for further analysis

In [89]:
# Lets check our dataset for any Null Values
df_merged.isna().sum()

ISIN            0
Date            0
Time            0
StartPrice      0
MaxPrice        0
MinPrice        0
EndPrice        0
TradedVolume    0
dtype: int64

## Data Transformation

In [90]:
# Lets check the dataframe to see whether we have any opening_price for the day
df_merged[(df_merged['ISIN'] == 'AT0000A0E9W5') & (df_merged['Date']== '2022-01-03')].sort_values(['Time'])

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2022-01-03,08:00,14.76,14.76,14.75,14.75,4414
157,AT0000A0E9W5,2022-01-03,08:01,14.80,14.83,14.80,14.83,2214
266,AT0000A0E9W5,2022-01-03,08:02,14.85,14.87,14.85,14.85,1634
572,AT0000A0E9W5,2022-01-03,08:03,14.85,14.85,14.85,14.85,1110
3045,AT0000A0E9W5,2022-01-03,08:05,14.82,14.82,14.80,14.80,5690
...,...,...,...,...,...,...,...,...
112131,AT0000A0E9W5,2022-01-03,16:24,15.63,15.63,15.63,15.63,198
112418,AT0000A0E9W5,2022-01-03,16:25,15.62,15.62,15.60,15.60,1268
113617,AT0000A0E9W5,2022-01-03,16:29,15.64,15.65,15.64,15.65,691
114130,AT0000A0E9W5,2022-01-03,16:35,15.62,15.62,15.62,15.62,28173


* We can see that there is no column which provides us details about the opening price for a particular date
* Now in the next steps we will create a few columns like opening_price, closing_price etc, which will be a part of our data transformation process.

### Get Opening price per ISIN per day

In [91]:
# Creating a new column opening_price, which will be equal to the first reported StartPrice per ISIN per Day
df_merged['opening_price'] = df_merged.sort_values(['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
df_merged[(df_merged['ISIN'] == 'AT0000A0E9W5')]

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,AT0000A0E9W5,2022-01-03,08:00,14.76,14.76,14.75,14.75,4414,14.76
157,AT0000A0E9W5,2022-01-03,08:01,14.80,14.83,14.80,14.83,2214,14.76
266,AT0000A0E9W5,2022-01-03,08:02,14.85,14.87,14.85,14.85,1634,14.76
572,AT0000A0E9W5,2022-01-03,08:03,14.85,14.85,14.85,14.85,1110,14.76
3045,AT0000A0E9W5,2022-01-03,08:05,14.82,14.82,14.80,14.80,5690,14.76
...,...,...,...,...,...,...,...,...,...
234495,AT0000A0E9W5,2022-01-04,16:26,15.00,15.03,15.00,15.01,9731,15.66
234779,AT0000A0E9W5,2022-01-04,16:27,15.00,15.00,15.00,15.00,296,15.66
235095,AT0000A0E9W5,2022-01-04,16:28,15.00,15.05,15.00,15.03,10339,15.66
235443,AT0000A0E9W5,2022-01-04,16:29,15.00,15.01,15.00,15.00,1144,15.66


* Now we have a column which shows the opening price per ISIN per day.
* We will now create some more columns in the same fashion which are needed for further analysis

### Get Closing price per ISIN per day

In [92]:
# Creating a new column named closing_price
df_merged['closing_price'] = df_merged.sort_values(['Time']).groupby(['ISIN', 'Date'])['EndPrice'].transform('last')
df_merged[(df_merged['ISIN'] == 'AT0000A0E9W5')]

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
0,AT0000A0E9W5,2022-01-03,08:00,14.76,14.76,14.75,14.75,4414,14.76,15.62
157,AT0000A0E9W5,2022-01-03,08:01,14.80,14.83,14.80,14.83,2214,14.76,15.62
266,AT0000A0E9W5,2022-01-03,08:02,14.85,14.87,14.85,14.85,1634,14.76,15.62
572,AT0000A0E9W5,2022-01-03,08:03,14.85,14.85,14.85,14.85,1110,14.76,15.62
3045,AT0000A0E9W5,2022-01-03,08:05,14.82,14.82,14.80,14.80,5690,14.76,15.62
...,...,...,...,...,...,...,...,...,...,...
234495,AT0000A0E9W5,2022-01-04,16:26,15.00,15.03,15.00,15.01,9731,15.66,14.96
234779,AT0000A0E9W5,2022-01-04,16:27,15.00,15.00,15.00,15.00,296,15.66,14.96
235095,AT0000A0E9W5,2022-01-04,16:28,15.00,15.05,15.00,15.03,10339,15.66,14.96
235443,AT0000A0E9W5,2022-01-04,16:29,15.00,15.01,15.00,15.00,1144,15.66,14.96


### Aggregations

In [93]:
df_merged = df_merged.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur = ('closing_price', 'min'), mimimum_price_eur = ('MinPrice','min'), maximum_price_eur = ('MaxPrice','max'), daily_traded_volume =('TradedVolume','sum'))

In [94]:
df_merged

,ISIN,Date,opening_price_eur,closing_price_eur,mimimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT000000STR1,2022-01-03,36.550,37.400,36.3500,37.400,660
1,AT000000STR1,2022-01-04,37.750,37.850,37.7500,37.850,27
2,AT00000FACC2,2022-01-03,7.390,7.650,7.3900,7.670,1045
3,AT00000FACC2,2022-01-04,7.860,7.790,7.7900,7.950,681
4,AT0000606306,2022-01-03,25.800,25.860,25.8000,26.000,455
...,...,...,...,...,...,...,...
6348,XS2314659447,2022-01-04,8.495,8.548,8.4555,8.548,0
6349,XS2314660700,2022-01-03,17.967,17.867,17.8670,18.097,19
6350,XS2314660700,2022-01-04,18.305,18.606,18.3050,18.606,0
6351,XS2376095068,2022-01-03,41.504,41.044,41.0000,41.832,2


* After the above transformations we have:
    1. Opening Price Column for the day
    2. Closing price column for the day
    3. maximum price column for the day
    4. minimum price column for the day
    5. all the above columns show amount in Euros
    6. Daily traded volumn for a particular stock for a particular day 

### Percent change previous closing

In [95]:
# creating a new column previous_closiong_price and assigning the value of closing_price_eur column of previous day
df_merged['previous_closing_price'] = df_merged.sort_values(['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

# Calculating precent change from previous day's closing price
df_merged['change_in_closing_in_%'] = round((df_merged['closing_price_eur'] - df_merged['previous_closing_price'])/df_merged['previous_closing_price']*100,2)



,ISIN,Date,opening_price_eur,closing_price_eur,mimimum_price_eur,maximum_price_eur,daily_traded_volume,previous_closing_price,change_in_closing_in_%
0,AT000000STR1,2022-01-03,36.550,37.400,36.3500,37.400,660,NaN,NaN
1,AT000000STR1,2022-01-04,37.750,37.850,37.7500,37.850,27,37.4000,1.20
2,AT00000FACC2,2022-01-03,7.390,7.650,7.3900,7.670,1045,NaN,NaN
3,AT00000FACC2,2022-01-04,7.860,7.790,7.7900,7.950,681,7.6500,1.83
4,AT0000606306,2022-01-03,25.800,25.860,25.8000,26.000,455,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6348,XS2314659447,2022-01-04,8.495,8.548,8.4555,8.548,0,8.6615,-1.31
6349,XS2314660700,2022-01-03,17.967,17.867,17.8670,18.097,19,NaN,NaN
6350,XS2314660700,2022-01-04,18.305,18.606,18.3050,18.606,0,17.8670,4.14
6351,XS2376095068,2022-01-03,41.504,41.044,41.0000,41.832,2,NaN,NaN


In [97]:
# Dropping column previous_closing_price as it is no longer needed
df_merged.drop(columns = ['previous_closing_price'], inplace = True)
df_merged

,ISIN,Date,opening_price_eur,closing_price_eur,mimimum_price_eur,maximum_price_eur,daily_traded_volume,change_in_closing_in_%
0,AT000000STR1,2022-01-03,36.550,37.400,36.3500,37.400,660,NaN
1,AT000000STR1,2022-01-04,37.750,37.850,37.7500,37.850,27,1.20
2,AT00000FACC2,2022-01-03,7.390,7.650,7.3900,7.670,1045,NaN
3,AT00000FACC2,2022-01-04,7.860,7.790,7.7900,7.950,681,1.83
4,AT0000606306,2022-01-03,25.800,25.860,25.8000,26.000,455,NaN
...,...,...,...,...,...,...,...,...
6348,XS2314659447,2022-01-04,8.495,8.548,8.4555,8.548,0,-1.31
6349,XS2314660700,2022-01-03,17.967,17.867,17.8670,18.097,19,NaN
6350,XS2314660700,2022-01-04,18.305,18.606,18.3050,18.606,0,4.14
6351,XS2376095068,2022-01-03,41.504,41.044,41.0000,41.832,2,NaN


In [98]:
# Lets round every number to upto two decimals
df_merged = df_merged.round(decimals = 2)
df_merged

,ISIN,Date,opening_price_eur,closing_price_eur,mimimum_price_eur,maximum_price_eur,daily_traded_volume,change_in_closing_in_%
0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
2,AT00000FACC2,2022-01-03,7.39,7.65,7.39,7.67,1045,NaN
3,AT00000FACC2,2022-01-04,7.86,7.79,7.79,7.95,681,1.83
4,AT0000606306,2022-01-03,25.80,25.86,25.80,26.00,455,NaN
...,...,...,...,...,...,...,...,...
6348,XS2314659447,2022-01-04,8.49,8.55,8.46,8.55,0,-1.31
6349,XS2314660700,2022-01-03,17.97,17.87,17.87,18.10,19,NaN
6350,XS2314660700,2022-01-04,18.30,18.61,18.30,18.61,0,4.14
6351,XS2376095068,2022-01-03,41.50,41.04,41.00,41.83,2,NaN
